In [ ]:
# from flask import Flask, request, jsonify
# import numpy as np
# import random
# import math
# import time
# import matplotlib.pyplot as plt

# app = Flask(__name__)

# # Constants for ACO
# ALPHA = 1.0
# BETA = 5.0
# EVAPORATION_RATE = 0.5
# Q = 100
# NUM_ANTS = 10
# NUM_ITERATIONS = 100

# # Distance calculation (Haversine formula)
# def calculate_distance(node1, node2):
#     if node1 is None or node2 is None:
#         raise ValueError("Node1 or Node2 is None!")

#     R = 6371  # Earth radius in kilometers
#     lat1, lon1 = math.radians(node1.latitude), math.radians(node1.longitude)
#     lat2, lon2 = math.radians(node2.latitude), math.radians(node2.longitude)
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
#     return R * c

# # GraphNode class for geographical info
# class GraphNode:
#     def __init__(self, id, latitude, longitude):
#         self.id = id
#         self.latitude = latitude
#         self.longitude = longitude

# class Driver(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# class Vendor(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# class Shop(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# # Ant class representing one ant in the ACO algorithm
# class Ant:
#     def __init__(self, driver, vendors, shops, pheromone_matrix, distance_matrix):
#         self.driver = driver
#         self.vendors = vendors
#         self.shops = shops
#         self.pheromone_matrix = pheromone_matrix
#         self.distance_matrix = distance_matrix
#         self.visited_nodes = []
#         self.current_position = driver
#         self.total_distance = 0.0

#     def select_next_node(self):
#         # Prioritize visiting vendors first, then shops
#         unvisited_nodes = ([self.driver] + self.vendors + self.shops)
#         unvisited_nodes = [node for node in unvisited_nodes if node not in self.visited_nodes]

#         if not unvisited_nodes:
#             raise Exception("All nodes have been visited!")

#         probabilities = []
#         for node in unvisited_nodes:
#             pheromone = self.pheromone_matrix[self.current_position.id][node.id]
#             distance = self.distance_matrix[self.current_position.id][node.id]

#             if distance == 0:  # Prevent division by zero
#                 continue

#             prob = (pheromone * ALPHA) * ((1.0 / distance) ** BETA)
#             probabilities.append(prob)

#         # Normalize probabilities
#         total_prob = sum(probabilities)
#         if total_prob == 0:  # Prevent invalid value encountered
#             raise Exception("No valid probabilities for selection")

#         probabilities = [prob / total_prob for prob in probabilities]

#         # Roulette wheel selection
#         r = random.uniform(0, 1)
#         cumulative_prob = 0.0
#         for i, node in enumerate(unvisited_nodes):
#             cumulative_prob += probabilities[i]
#             if r <= cumulative_prob:
#                 return node

#     def tour(self):
#         self.visited_nodes.append(self.current_position)

#         # Visit all vendors first
#         for vendor in self.vendors:
#             if vendor not in self.visited_nodes:
#                 self.total_distance += calculate_distance(self.current_position, vendor)
#                 self.current_position = vendor
#                 self.visited_nodes.append(vendor)

#         # Now visit all shops
#         for shop in self.shops:
#             if shop not in self.visited_nodes:
#                 self.total_distance += calculate_distance(self.current_position, shop)
#                 self.current_position = shop
#                 self.visited_nodes.append(shop)

#         self.total_distance += calculate_distance(self.current_position, self.driver)
#         return self.visited_nodes, self.total_distance

# # ACO Algorithm implementation with performance metrics
# def aco_routing(driver, vendors, shops):
#     num_nodes = len(vendors) + len(shops) + 1
#     pheromone_matrix = np.ones((num_nodes, num_nodes))
#     distance_matrix = np.zeros((num_nodes, num_nodes))

#     nodes = [driver] + vendors + shops
#     for i in range(num_nodes):
#         for j in range(i + 1, num_nodes):
#             distance = calculate_distance(nodes[i], nodes[j])
#             distance_matrix[i][j] = distance
#             distance_matrix[j][i] = distance

#     best_distance = float('inf')
#     best_tour = None
#     distance_history = []  # To track the best distance found over iterations
#     start_time = time.time()  # Start time for performance measurement

#     for iteration in range(NUM_ITERATIONS):
#         ants = [Ant(driver, vendors, shops, pheromone_matrix, distance_matrix) for _ in range(NUM_ANTS)]
#         for ant in ants:
#             tour, distance = ant.tour()
#             if distance < best_distance:
#                 best_distance = distance
#                 best_tour = tour

#         distance_history.append(best_distance)  # Track the best distance found

#         pheromone_matrix *= (1 - EVAPORATION_RATE)
#         for ant in ants:
#             for i in range(len(ant.visited_nodes) - 1):
#                 start_node = ant.visited_nodes[i]
#                 end_node = ant.visited_nodes[i + 1]
#                 pheromone_matrix[start_node.id][end_node.id] += Q / ant.total_distance
#                 pheromone_matrix[end_node.id][start_node.id] += Q / ant.total_distance

#     end_time = time.time()  # End time for performance measurement
#     elapsed_time = end_time - start_time

#     # Plotting the distance history
#     plt.plot(distance_history)
#     plt.title('Best Distance Found Over Iterations')
#     plt.xlabel('Iteration')
#     plt.ylabel('Distance')
#     plt.savefig('aco_distance_history.png')  # Save the plot
#     plt.close()  # Close the plot to avoid display issues in Flask

#     return best_tour, best_distance, elapsed_time

# # Function to calculate routes using ACO
# def get_aco_paths(driver, vendors, shops):
#     best_tour, best_distance, elapsed_time = aco_routing(driver, vendors, shops)

#     # Reorder the path to include the driver first, followed by vendors and shops
#     reordered_path = [driver.name] + [vendor.name for vendor in vendors] + [shop.name for shop in best_tour[len(vendors):]]

#     # Generate Google Maps link for the best tour
#     waypoints = "|".join([f"{node.latitude},{node.longitude}" for node in best_tour])
#     maps_link = f"https://www.google.com/maps/dir/?api=1&origin={driver.latitude},{driver.longitude}&destination={waypoints}&waypoints={waypoints}&travelmode=driving"

#     return {
#         "distance": best_distance,
#         "path": reordered_path,
#         "maps_link": maps_link,
#         "time_taken": elapsed_time
#     }

# @app.route('/api/post-data', methods=['POST'])
# def post_data():
#     data = request.get_json()

#     try:
#         driver = Driver(0, "Current", data['driver'][0]['Latitude'], data['driver'][0]['Longitude'])
#         vendors = [Vendor(v['id'], v['VendorName'], v['Latitude'], v['Longitude']) for v in data['vendors']]
#         shops = [Shop(s['id'], s['Name'], s['Latitude'], s['Longitude']) for s in data['shops']]
        
#         if not vendors or not shops:
#             return jsonify({'error': 'Vendors and shops must not be empty.'}), 400
            
#     except KeyError as e:
#         return jsonify({'error': f'Missing key: {str(e)}'}), 400

#     path = get_aco_paths(driver, vendors, shops)

#     return jsonify({
#         'message': 'Data received successfully!',
#         'path': path
#     }), 200

# if __name__ == '__main__':
#     app.run()


In [ ]:
# from flask import Flask, request, jsonify
# import numpy as np
# import random
# import math
# import time
# import matplotlib.pyplot as plt

# app = Flask(__name__)

# # Constants for ACO
# ALPHA = 1.0
# BETA = 5.0
# EVAPORATION_RATE = 0.5
# Q = 100
# NUM_ANTS = 10
# NUM_ITERATIONS = 100

# # Distance calculation (Haversine formula)
# def calculate_distance(node1, node2):
#     if node1 is None or node2 is None:
#         raise ValueError("Node1 or Node2 is None!")

#     R = 6371  # Earth radius in kilometers
#     lat1, lon1 = math.radians(node1.latitude), math.radians(node1.longitude)
#     lat2, lon2 = math.radians(node2.latitude), math.radians(node2.longitude)
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
#     return R * c

# # GraphNode class for geographical info
# class GraphNode:
#     def __init__(self, id, latitude, longitude):
#         self.id = id
#         self.latitude = latitude
#         self.longitude = longitude

# class Driver(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# class Vendor(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# class Shop(GraphNode):
#     def __init__(self, id, name, latitude, longitude):
#         super().__init__(id, latitude, longitude)
#         self.name = name

# # Ant class representing one ant in the ACO algorithm
# class Ant:
#     def __init__(self, driver, vendors, shops, pheromone_matrix, distance_matrix):
#         self.driver = driver
#         self.vendors = vendors
#         self.shops = shops
#         self.pheromone_matrix = pheromone_matrix
#         self.distance_matrix = distance_matrix
#         self.visited_nodes = []
#         self.current_position = driver
#         self.total_distance = 0.0

#     def select_next_node(self):
#         # Prioritize visiting vendors first, then shops
#         unvisited_nodes = ([self.driver] + self.vendors + self.shops)
#         unvisited_nodes = [node for node in unvisited_nodes if node not in self.visited_nodes]

#         if not unvisited_nodes:
#             raise Exception("All nodes have been visited!")

#         probabilities = []
#         for node in unvisited_nodes:
#             pheromone = self.pheromone_matrix[self.current_position.id][node.id]
#             distance = self.distance_matrix[self.current_position.id][node.id]

#             if distance == 0:  # Prevent division by zero
#                 continue

#             prob = (pheromone * ALPHA) * ((1.0 / distance) ** BETA)
#             probabilities.append(prob)

#         # Normalize probabilities
#         total_prob = sum(probabilities)
#         if total_prob == 0:  # Prevent invalid value encountered
#             raise Exception("No valid probabilities for selection")

#         probabilities = [prob / total_prob for prob in probabilities]

#         # Roulette wheel selection
#         r = random.uniform(0, 1)
#         cumulative_prob = 0.0
#         for i, node in enumerate(unvisited_nodes):
#             cumulative_prob += probabilities[i]
#             if r <= cumulative_prob:
#                 return node

#     def tour(self):
#         self.visited_nodes.append(self.current_position)

#         # Visit all vendors first
#         for vendor in self.vendors:
#             if vendor not in self.visited_nodes:
#                 self.total_distance += calculate_distance(self.current_position, vendor)
#                 self.current_position = vendor
#                 self.visited_nodes.append(vendor)

#         # Now visit all shops
#         for shop in self.shops:
#             if shop not in self.visited_nodes:
#                 self.total_distance += calculate_distance(self.current_position, shop)
#                 self.current_position = shop
#                 self.visited_nodes.append(shop)

#         # Return to the driver location
#         self.total_distance += calculate_distance(self.current_position, self.driver)
#         self.visited_nodes.append(self.driver)  # Ensure the driver is also marked as visited

#         return self.visited_nodes, self.total_distance

# # ACO Algorithm implementation with performance metrics
# def aco_routing(driver, vendors, shops):
#     num_nodes = len(vendors) + len(shops) + 1
#     pheromone_matrix = np.ones((num_nodes, num_nodes))
#     distance_matrix = np.zeros((num_nodes, num_nodes))

#     nodes = [driver] + vendors + shops
#     for i in range(num_nodes):
#         for j in range(i + 1, num_nodes):
#             distance = calculate_distance(nodes[i], nodes[j])
#             distance_matrix[i][j] = distance
#             distance_matrix[j][i] = distance

#     best_distance = float('inf')
#     best_tour = None
#     distance_history = []  # To track the best distance found over iterations
#     start_time = time.time()  # Start time for performance measurement

#     for iteration in range(NUM_ITERATIONS):
#         ants = [Ant(driver, vendors, shops, pheromone_matrix, distance_matrix) for _ in range(NUM_ANTS)]
#         for ant in ants:
#             tour, distance = ant.tour()
#             if distance < best_distance:
#                 best_distance = distance
#                 best_tour = tour

#         distance_history.append(best_distance)  # Track the best distance found

#         pheromone_matrix *= (1 - EVAPORATION_RATE)
#         for ant in ants:
#             for i in range(len(ant.visited_nodes) - 1):
#                 start_node = ant.visited_nodes[i]
#                 end_node = ant.visited_nodes[i + 1]
#                 pheromone_matrix[start_node.id][end_node.id] += Q / ant.total_distance
#                 pheromone_matrix[end_node.id][start_node.id] += Q / ant.total_distance

#     end_time = time.time()  # End time for performance measurement
#     elapsed_time = end_time - start_time

#     # Plotting the distance history
#     plt.plot(distance_history)
#     plt.title('Best Distance Found Over Iterations')
#     plt.xlabel('Iteration')
#     plt.ylabel('Distance')
#     plt.savefig('aco_distance_history.png')  # Save the plot
#     plt.close()  # Close the plot to avoid display issues in Flask

#     return best_tour, best_distance, elapsed_time

# # Function to calculate routes using ACO
# def get_aco_paths(driver, vendors, shops):
#     best_tour, best_distance, elapsed_time = aco_routing(driver, vendors, shops)

#     # Reorder the path to include the driver first, followed by vendors and shops
#     reordered_path = [driver.name] + [vendor.name for vendor in vendors] + [shop.name for shop in best_tour[len(vendors):]]

#     # Generate Google Maps link for the best tour
#     waypoints = "|".join([f"{node.latitude},{node.longitude}" for node in best_tour])
#     maps_link = f"https://www.google.com/maps/dir/?api=1&origin={driver.latitude},{driver.longitude}&destination={waypoints}&waypoints={waypoints}&travelmode=driving"

#     return {
#         "distance": best_distance,
#         "path": reordered_path,
#         "maps_link": maps_link,
#         "time_taken": elapsed_time
#     }

# @app.route('/api/post-data', methods=['POST'])
# def post_data():
#     data = request.get_json()

#     try:
#         driver = Driver(0, "Current", data['driver'][0]['Latitude'], data['driver'][0]['Longitude'])
#         vendors = [Vendor(v['id'], v['VendorName'], v['Latitude'], v['Longitude']) for v in data['vendors']]
#         shops = [Shop(s['id'], s['Name'], s['Latitude'], s['Longitude']) for s in data['shops']]
        
#         if not vendors or not shops:
#             return jsonify({'error': 'Vendors and shops must not be empty.'}), 400
            
#     except KeyError as e:
#         return jsonify({'error': f'Missing key: {str(e)}'}), 400

#     # First calculate the optimized route
#     path = get_aco_paths(driver, vendors, shops)

#     return jsonify({
#         'message': 'Data received successfully!',
#         'path': path
#     }), 200

# if __name__ == '__main__':
#     app.run()


In [ ]:
# this is okay algorithm

from flask import Flask, request, jsonify
import numpy as np
import random
import math
import time
import matplotlib.pyplot as plt

app = Flask(__name__)

# Constants for ACO
ALPHA = 1.0
BETA = 5.0
EVAPORATION_RATE = 0.5
Q = 100
NUM_ANTS = 10
NUM_ITERATIONS = 100

# Distance calculation (Haversine formula)
def calculate_distance(node1, node2):
    if node1 is None or node2 is None:
        raise ValueError("Node1 or Node2 is None!")

    R = 6371  # Earth radius in kilometers
    lat1, lon1 = math.radians(node1.latitude), math.radians(node1.longitude)
    lat2, lon2 = math.radians(node2.latitude), math.radians(node2.longitude)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# GraphNode class for geographical info
class GraphNode:
    def __init__(self, id, latitude, longitude):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude

class Driver(GraphNode):
    def __init__(self, id, name, latitude, longitude):
        super().__init__(id, latitude, longitude)
        self.name = name

class Vendor(GraphNode):
    def __init__(self, id, name, latitude, longitude):
        super().__init__(id, latitude, longitude)
        self.name = name

class Shop(GraphNode):
    def __init__(self, id, name, latitude, longitude):
        super().__init__(id, latitude, longitude)
        self.name = name

# Ant class representing one ant in the ACO algorithm
class Ant:
    def __init__(self, driver, vendors, shops, pheromone_matrix, distance_matrix):
        self.driver = driver
        self.vendors = vendors
        self.shops = shops
        self.pheromone_matrix = pheromone_matrix
        self.distance_matrix = distance_matrix
        self.visited_nodes = []
        self.current_position = driver
        self.total_distance = 0.0

    def select_next_node(self):
        unvisited_nodes = [self.driver] + self.vendors + self.shops
        unvisited_nodes = [node for node in unvisited_nodes if node not in self.visited_nodes]

        if not unvisited_nodes:
            raise Exception("All nodes have been visited!")

        probabilities = []
        for node in unvisited_nodes:
            pheromone = self.pheromone_matrix[self.current_position.id][node.id]
            distance = self.distance_matrix[self.current_position.id][node.id]

            if distance == 0:  # Prevent division by zero
                continue

            prob = (pheromone * ALPHA) * ((1.0 / distance) ** BETA)
            probabilities.append(prob)

        # Normalize probabilities
        total_prob = sum(probabilities)
        if total_prob == 0:  # Prevent invalid value encountered
            raise Exception("No valid probabilities for selection")

        probabilities = [prob / total_prob for prob in probabilities]

        # Roulette wheel selection
        r = random.uniform(0, 1)
        cumulative_prob = 0.0
        for i, node in enumerate(unvisited_nodes):
            cumulative_prob += probabilities[i]
            if r <= cumulative_prob:
                return node

    def tour(self):
        self.visited_nodes.append(self.current_position)

        # Visit all vendors first
        for vendor in self.vendors:
            if vendor not in self.visited_nodes:
                self.total_distance += calculate_distance(self.current_position, vendor)
                self.current_position = vendor
                self.visited_nodes.append(vendor)

        # Now visit all shops
        for shop in self.shops:
            if shop not in self.visited_nodes:
                self.total_distance += calculate_distance(self.current_position, shop)
                self.current_position = shop
                self.visited_nodes.append(shop)

        self.total_distance += calculate_distance(self.current_position, self.driver)
        return self.visited_nodes, self.total_distance

# ACO Algorithm implementation
def aco_routing(driver, vendors, shops):
    num_nodes = len(vendors) + len(shops) + 1
    pheromone_matrix = np.ones((num_nodes, num_nodes))
    distance_matrix = np.zeros((num_nodes, num_nodes))

    nodes = [driver] + vendors + shops
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            distance = calculate_distance(nodes[i], nodes[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance

    best_distance = float('inf')
    best_tour = None
    distance_history = []  # To track the best distance found over iterations

    for iteration in range(NUM_ITERATIONS):
        ants = [Ant(driver, vendors, shops, pheromone_matrix, distance_matrix) for _ in range(NUM_ANTS)]
        for ant in ants:
            tour, distance = ant.tour()
            if distance < best_distance:
                best_distance = distance
                best_tour = tour

        distance_history.append(best_distance)

        pheromone_matrix *= (1 - EVAPORATION_RATE)
        for ant in ants:
            for i in range(len(ant.visited_nodes) - 1):
                start_node = ant.visited_nodes[i]
                end_node = ant.visited_nodes[i + 1]
                pheromone_matrix[start_node.id][end_node.id] += Q / ant.total_distance
                pheromone_matrix[end_node.id][start_node.id] += Q / ant.total_distance

    # Plotting the distance history
    plt.plot(distance_history)
    plt.title('Best Distance Found Over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.savefig('aco_distance_history.png')  # Save the plot
    plt.close()  # Close the plot to avoid display issues in Flask

    return best_tour, best_distance

# Function to calculate routes using ACO
def get_aco_paths(driver, vendors, shops):
    best_tour, best_distance = aco_routing(driver, vendors, shops)

    # Reorder the path to include the driver first, followed by vendors and shops
    reordered_path = [driver.name] + [vendor.name for vendor in vendors] + [shop.name for shop in best_tour[len(vendors):]]

    # Generate Google Maps link for the best tour after finding the optimal path
    waypoints = "/".join([f"{node.latitude},{node.longitude}" for node in best_tour])
    maps_link = f"https://www.google.com/maps/dir/{waypoints}"

    return {
        "distance": best_distance,
        "path": reordered_path,
        "maps_link": maps_link
    }

@app.route('/api/post-data', methods=['POST'])
def post_data():
    data = request.get_json()

    try:
        driver = Driver(0, "Current", data['driver'][0]['Latitude'], data['driver'][0]['Longitude'])
        vendors = [Vendor(v['id'], v['VendorName'], v['Latitude'], v['Longitude']) for v in data['vendors']]
        shops = [Shop(s['id'], s['Name'], s['Latitude'], s['Longitude']) for s in data['shops']]
        
        if not vendors or not shops:
            return jsonify({'error': 'Vendors and shops must not be empty.'}), 400
            
    except KeyError as e:
        return jsonify({'error': f'Missing key: {str(e)}'}), 400

    # Get optimal paths and corresponding details
    path = get_aco_paths(driver, vendors, shops)

    return jsonify({
        'message': 'Data received successfully!',
        'path': path
    }), 200

if __name__ == '__main__':
    app.run()
